# import new/updated characteristics from google sheet file exported as csv


In [27]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

# import image urls from html export


In [11]:
name_img_url = {}

In [12]:
from bs4 import BeautifulSoup
f=open('20200608 SEARCH-by strat. representation-Bin. obs_v1/MiCorr.html')
content=f.read()
soup=BeautifulSoup(content,'html.parser')
imgs=soup.find_all('img')
name_img_url={}
for im in imgs:
    parent_td=im.parent.parent if im.parent.parent.name=='td' else None
    if parent_td:
        name_img_url[parent_td.previous_sibling.previous_sibling.text]=im['src']

In [254]:

def new_characteristic(family_uid, uid, name, order, description=None, image_url=None, optgroup=None, natures=[]):
    cql_update_row = f"""{{family_uid:'{family_uid}', uid:'{uid}', name:'{name}', order:{order}, natures:{natures}"""
    arg_list=[cql_update_row]

    if image_url is None:
        # search characteristic name in name to image_url map
        image_url = name_img_url.get(name)
    if description:
        arg_list.append(f"description:'{description}'")
    if image_url:
        arg_list.append(f"image_url:'{image_url}'")
    if optgroup:
        arg_list.append(f"optgroup:'{optgroup}'")

    return ', '.join(arg_list) + '},'

def new_family(fg_uid, uid, name, order, observation, description='',visible=False, variable=False, unit=None, fieldset=None,natures=[], list_elem_family=None, upto=None):
    # adapt csv column value to cql format
    observation=observation.lower()
    if observation=='cross-section':
        observation='cross_section'
    cql_update_row = f"""{{fg:'{fg_uid}', order:{order}, observation:{observation}, uid:'{uid}', name:'{name}', visible: {visible}, variable: {variable}, description:'{description}', natures:{natures}"""
    arg_list=[cql_update_row]
    if unit:
        arg_list.append(f"unit:'{unit}'")
    if fieldset:
        arg_list.append(f"fieldset:'{fieldset}'")
    if list_elem_family:
        arg_list.append(f"list_elem_family:'{list_elem_family}'")
    if upto is not None:
        arg_list.append(f"upto:{upto}")
    return ', '.join(arg_list) + '},'

def new_subcharacteristic(parent_uid, uid, name, order, variable=False, description=None, natures=[], unit=None, image_url=None, optgroup=None, list_elem_family=None, upto=None):
    cql_update_row = f"""{{parent_uid:'{parent_uid}', uid:'{uid}', name:'{name}', variable:{variable}, order:{order}, natures:{natures}"""
    arg_list=[cql_update_row]
    if unit:
        arg_list.append(f"unit:'{unit}'")
    if image_url is None:
        # search characteristic name in name to image_url map
        image_url = name_img_url.get(name)
    if description:
        arg_list.append(f"description:'{description}'")
    if image_url:
        arg_list.append(f"image_url:'{image_url}'")
    if optgroup:
        arg_list.append(f"optgroup:'{optgroup}'")
    if list_elem_family:
        arg_list.append(f"list_elem_family:'{list_elem_family}'")
    if upto is not None:
        arg_list.append(f"upto:{upto}")

    return ', '.join(arg_list) + '},'


In [14]:
print(new_subcharacteristic('x','y','z',1))

{parent_uid:'x', uid:'y', name:'z', variable:False, order:1, natures:[]},


## Microstructure Binocular special case

In [49]:

# we treat all values and sub-values as characteristics (with optgroup + property for Cristalline microstructure)
# and optgroup - (closure of previous group) for isolated aggregate


with open(csv_file_path,"r") as f:
    reader =csv.DictReader(f)
    current_value=''
    characteristic_order = 1
    family_order = 1
    sub_value_order = 1
    optgroup_level = 0
    description_field_in = {'Binocular':'Definition from observation under binocular',
                           'Cross-section': 'Definition from observation on cross-section'}

    for r in reader:
        if 'Microstructure' not in r['Characteristic']:
            continue
        observation = r['Observation mode']
        if observation != 'Binocular':
            continue

        if r['Values'] == '' and r['Sub-values'] == '':
            family_uid='microstructureFamily'
            cql_family_update_row=new_family(fg_uid='fgMicrostructure', uid=family_uid, name='Microstructure',
                                     order=family_order,observation='binocular', description=r[description_field_in[observation]])
            print(cql_family_update_row)
            print(f'\n// {family_uid} characteristics:')
            family_order+=1

        if r['Values'] != '':
            # print('\nNew characteristic:\n')
            current_value = r['Values']
            current_value_uid = to_uid(r['Characteristic'] + ' ' +current_value)+'Characteristic'
            # trick to detect end of optgroup in our particular case (1 optgroup only yet)
            if optgroup_level:
                optgroup=None
                optgroup_level-=1

            # print(new_characteristic(family_uid=family_uid, uid=current_value_uid,
            #                         name=current_value,order=characteristic_order,
            #                         description=r[description_field_in[observation]],optgroup=optgroup))
            # family_order+=1
            # characteristic_order = 1
            characteristic_order += 1

        # print(observation, r['Characteristic'], current_value,'None' if r['Sub-values'] == '' else r['Sub-values'])
        current_subvalue = r['Sub-values']
        if current_value!='' and current_subvalue!='':
            if sub_value_order==1:
                optgroup_level+=1
                optgroup=current_value
                print(f'// {current_value_uid} optgroup options:')
            current_subvalue_uid = to_uid(current_value + ' ' +current_subvalue)+'Characteristic'
            if observation=='Binocular':
                print(new_characteristic(family_uid=family_uid, uid=current_subvalue_uid,name=current_subvalue,
                                     order=characteristic_order))
            else:
                print(new_characteristic(family_uid=family_uid, uid=current_subvalue_uid,name=current_subvalue,
                                     order=characteristic_order))

            sub_value_order+=1
            characteristic_order+=1

{fg:'fgMicrostructure', order:1, observation:binocular, uid:'microstructureFamily', name:'Microstructure', visible: False, variable: False, description:'Assessed on the observed area. The blank option corresponds to a stratum which microstructure is not known.'},

// microstructureFamily characteristics:
// microstructureCristallineMicrostructureCharacteristic optgroup options:
{family_uid:'microstructureFamily', uid:'cristallineMicrostructureBlankCharacteristic', name:'Blank', order:4},
{family_uid:'microstructureFamily', uid:'cristallineMicrostructureDendriticCharacteristic', name:'Dendritic', order:5, image_url:'https://lh6.googleusercontent.com/FyP0N-Th3gRuKFukmtMQPekDzkTkk4rqsVoqWgZLep7-kvuIVy0tXhyBwC9BHm1TlRKAZdSNhPCAFfyNE-DmsKJbKz7YSqrn0SwjfX0WZbPeS_gJhDmiIizLf6osIuEfGqBLvBUXgA=w126-h119'},
{family_uid:'microstructureFamily', uid:'cristallineMicrostructureDrusyAggregateCharacteristic', name:'Drusy aggregate', order:6, image_url:'https://lh3.googleusercontent.com/i904_m1OrYim9EZy

In [267]:

FAMILY = 1
CHARACTERISTIC = 2
SUBCHARACTERISTIC = 4
DEBUG = 8
NATURES_L = ['CP','CM', 'D','M','NMM','POM','S','SV']
NATURES = {'CP':'cp','CM':'cm', 'D':'deposit','M':'metal','NMM':'nmm','POM':'pom','S':'soil','SV':'sv'}

NATURE_HEADER_SUFFIX = {'Binocular':' Binocular', 'Cross-section':' CS'}

family_uids=[]
characteristic_uids=[]

def suffix(observation, end):
    return ('CS' if observation=='Cross-section' else '') + end

def upto_list_elem(r):
        # element / compound list
    if r['Variables'] =='list':
        list_elem_family = r['unit']
        upto=0
    elif  r['Variables'] =='formula':
        list_elem_family = r['unit']
        upto=1                        
    else:                      
        list_elem_family = None
        upto = None
    return upto, list_elem_family

def import_family_group(family_group, filter_observation=None, fieldsets=[], filter_print=FAMILY|CHARACTERISTIC|SUBCHARACTERISTIC, shared_sub_characteristics=False):
    with open(csv_file_path,"r") as f:
        reader =csv.DictReader(f)
        current_value=''
        characteristic_order = 1
        family_order = 1
        sub_value_order = 1
        optgroup = None
        optgroup_level = 0
        description_field_in = {'Binocular':'Definition from observation under binocular',
                               'Cross-section': 'Definition from observation on cross-section'}
        fieldset = None # group Values below under fieldset

        for r in reader:
            if family_group not in r['Characteristic']:
                continue
            observation = r['Observation mode']
            if observation != filter_observation:
                continue
            # strata Nature filtering
            r_natures = [NATURES[N] for N in NATURES if r[N+NATURE_HEADER_SUFFIX[observation]]]
            if filter_print & DEBUG:
                print(f'\n{r}\n')
            # print(f'family_group={r["Characteristic"]}')
            # print(f'observation={observation}')
            if r['Values'] == '' and r['Sub-values'] == '' and r['Sub-characteristic']!='':
                family_name=r['Sub-characteristic']
                if '\n' in family_name:
                    # encode sub fieldset using code compatible '_' instead of '\n' that is used in source csv
                    family_name=family_name.replace('\n', '_')


                code_compatible_family_name = family_name.replace('(S)','s').replace('/ ','').replace('(s)','s').replace('_',' ')

                if family_name != r['Characteristic']: 
                    family_uid=to_uid(r['Characteristic'] + ' ' +code_compatible_family_name)+ suffix(observation, 'Family')
                else:
                    family_uid=to_uid(r['Characteristic'])+ suffix(observation, 'Family')
                if family_name in fieldsets:
                    fieldset = family_name
                    code_compatible_fieldset = fieldset.replace('(S)','s').replace('/ ','').replace('(s)','s').replace('_', ' ')
                    print(f'// new fieldset={fieldset}')
                else:
                    fieldset = None
                    code_compatible_fieldset = None
                if r['Variables']:
                    
                    # check if element / compound list
                    upto, list_elem_family = upto_list_elem(r)
                    
                    cql_family_update_row=new_family(fg_uid='fg'+family_group, uid=family_uid, name=family_name,
                                         order=family_order,observation=observation, description=r[description_field_in[observation]],
                                                    natures=r_natures, list_elem_family=list_elem_family,upto=upto)
                    family_uids.append(family_uid)
                    
                    if filter_print & FAMILY:
                        print(f'// new family_name={family_name} uid: {family_uid}')
                        print(cql_family_update_row)
                    ## print(f'\n// {family_uid} characteristics:')
                sub_value_order=1
                family_order+=1
            elif r['Values'] == '' and r['Sub-values'] != '':
                current_subvalue = r['Sub-values']
                if not shared_sub_characteristics:
                    current_subvalue_uid = to_uid(' '.join([r['Characteristic'], code_compatible_family_name, current_value, current_subvalue])) + suffix(observation, 'Characteristic')
                else:
                    current_subvalue_uid = to_uid(current_subvalue) + suffix(observation, 'Characteristic')                
                characteristic_uids.append(current_subvalue_uid)
                upto, list_elem_family = upto_list_elem(r)
                if not list_elem_family:
                    variable=r['Variables']!=''
                    unit=r['unit']
                else:
                    variable,unit=False,None
                cql_subcharacteristic_update_row=new_subcharacteristic(parent_uid=current_value_uid, uid=current_subvalue_uid,
                                                                       name=current_subvalue, order=sub_value_order,natures=r_natures,
                                                                      variable=variable, unit=unit,
                                                                      list_elem_family=list_elem_family,upto=upto)
                
                if filter_print & SUBCHARACTERISTIC:
                    # print(f'\n// {current_subvalue} sub-characteristic:')
                    print(cql_subcharacteristic_update_row)
                sub_value_order+=1

            if r['Values'] != '':
                current_value = r['Values']
                sub_value_order=1
                if r['Variables'] !='':
                    #print(f'\nNew {current_value} variable Family:\n')
                    upto, list_elem_family = upto_list_elem(r)
                    if not list_elem_family:
                        variable=r['Variables']!=''
                        unit=r['unit']
                    else:
                        variable,unit=False,None
                    current_value_uid = to_uid(' '.join([r['Characteristic'], (code_compatible_fieldset if fieldset else ''), current_value])) + suffix(observation, 'VarFamily' if variable else 'Family')                    
                    current_value_uid = current_value_uid.replace('(S)','s').replace('/ ','').replace('(s)','s')
                    family_uids.append(current_value_uid)
                    
                    cql_family_update_row=new_family(fg_uid='fg'+family_group, uid=current_value_uid, name=current_value,
                                order=family_order,observation=observation.lower(), description=r[description_field_in[observation]],
                                variable=variable, unit=unit, fieldset=fieldset, natures=r_natures,list_elem_family=list_elem_family,upto=upto)
                                                    
                    if filter_print & FAMILY:
                        print(cql_family_update_row)
                    family_order+=1
                else:
                    # print(f'\nNew {current_value} characteristic:\n')
                    current_value_uid = to_uid(' '.join([r['Characteristic'], code_compatible_family_name, current_value])) + suffix(observation, 'Characteristic')

                    characteristic_uids.append(current_value_uid)
                    
                    # trick to detect end of optgroup in our particular case (1 optgroup only yet)
                    if optgroup_level:
                        optgroup= None
                        optgroup_level-=1

                    cql_characteristic_update_row = new_characteristic(family_uid=family_uid, uid=current_value_uid,
                                         name=current_value,order=characteristic_order,
                                         description=r[description_field_in[observation]],optgroup=optgroup,natures=r_natures)
                    if filter_print & CHARACTERISTIC:
                        print(cql_characteristic_update_row)

                    # family_order+=1
                    # characteristic_order = 1
                    characteristic_order += 1
    if filter_print & DEBUG:
        print(f'family_uids={family_uids}\n len={len(family_uids)}\n set len={len(set(family_uids))}')
        for x in set(family_uids):
            family_uids.remove(x)
        print(f'duplicated items in family_uids={family_uids}')
        print(f'characteristic_uids={characteristic_uids}\n len={len(characteristic_uids)}\n set len={len(set(characteristic_uids))}')
        for x in set(characteristic_uids):
            characteristic_uids.remove(x)
        print(f'duplicated items in characteristic_uids={characteristic_uids}')
    



In [261]:
fs='''Second band / aggregate
Compounds'''
fs

'Second band / aggregate\nCompounds'

In [73]:
## Interface cross-section special case

In [42]:
# we treat Roughness as a fieldset (as it includes multiple families instead of characteristics)
# to do add group/fieldset column to spreadsheet

import_family_group('Interface', filter_observation='Cross-section', fieldsets=['Roughness'], filter_print=CHARACTERISTIC) #FAMILY|CHARACTERISTIC

{family_uid:'interfaceProfileCSFamily', uid:'interfaceBlankCSCharacteristic', name:'Blank', order:1, natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv']},
{family_uid:'interfaceProfileCSFamily', uid:'interfaceStraightCSCharacteristic', name:'Straight', order:2, natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv'], description:'No waves along the observed length.'},
{family_uid:'interfaceProfileCSFamily', uid:'interfaceWavyCSCharacteristic', name:'Wavy', order:3, natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv'], description:'Shows variations or undulations with a low height to length ratio. Angles are greater than 90°.'},
{family_uid:'interfaceProfileCSFamily', uid:'interfaceBumpyCSCharacteristic', name:'Bumpy', order:4, natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv'], description:'Presents variations that are close in height and length. Angles can be equal to or less than 90°.'},
{family_uid:'interfaceProfileCSFamily', uid:'interfaceIrr

In [24]:
print('\n //FAMILY \n')
import_family_group('Microstructure', filter_observation='Binocular', fieldsets=[], filter_print=FAMILY, shared_sub_characteristics=True) #FAMILY|CHARACTERISTI
print('\n //CHARACTERISTICS \n')
import_family_group('Microstructure', filter_observation='Binocular', fieldsets=[], filter_print=CHARACTERISTIC, shared_sub_characteristics=True) #FAMILY|CHARACTERISTI
print('\n //SUBCHARACTERISTICS \n')
import_family_group('Microstructure', filter_observation='Binocular', fieldsets=[], filter_print=SUBCHARACTERISTIC, shared_sub_characteristics=True) #FAMILY|CHARACTERISTI
3'


 //FAMILY 

// new family_name=Microstructure
{fg:'fgMicrostructure', order:1, observation:binocular, uid:'microstructureFamily', name:'Microstructure', visible: False, variable: False, description:'Assessed on the observed area. The blank option corresponds to a stratum which microstructure is not known., natures:['cp', 'cm', 'deposit', 'metal', 'nmm', 'pom', 'soil']'},

 //CHARACTERISTICS 

{family_uid:'microstructureFamily', uid:'microstructureBlankCharacteristic', name:'Blank', order:1, natures:['cp']},
{family_uid:'microstructureFamily', uid:'microstructureAlternativeBandsCharacteristic', name:'Alternative bands', order:2, natures:['cp'], description:'Occurrence of several alternately repeating layers of corrosion products of different composition.'},
{family_uid:'microstructureFamily', uid:'microstructureCristallineMicrostructureCharacteristic', name:'Cristalline microstructure', order:3, natures:['cp', 'deposit'], description:'Presence of a crystalline aggregates. The blank opt

In [16]:
# fixes SUBCHARACTERISTIC uid with shared_sub_characteristics = False like first generated characteristics
# in 
print('\n //SUBCHARACTERISTICS \n')
import_family_group('Microstructure', filter_observation='Binocular', fieldsets=[], filter_print=SUBCHARACTERISTIC, shared_sub_characteristics=False) #FAMILY|CHARACTERISTI



 //SUBCHARACTERISTICS 

{parent_uid:'microstructureCristallineMicrostructureCharacteristic', uid:'cristallineMicrostructureBlankCharacteristic', name:'Blank', variable:False, order:1, natures:[]},
{parent_uid:'microstructureCristallineMicrostructureCharacteristic', uid:'cristallineMicrostructureDendriticCharacteristic', name:'Dendritic', variable:False, order:2, natures:['cp', 'deposit'], image_url:'https://lh6.googleusercontent.com/FyP0N-Th3gRuKFukmtMQPekDzkTkk4rqsVoqWgZLep7-kvuIVy0tXhyBwC9BHm1TlRKAZdSNhPCAFfyNE-DmsKJbKz7YSqrn0SwjfX0WZbPeS_gJhDmiIizLf6osIuEfGqBLvBUXgA=w126-h119'},
{parent_uid:'microstructureCristallineMicrostructureCharacteristic', uid:'cristallineMicrostructureDrusyAggregateCharacteristic', name:'Drusy aggregate', variable:False, order:3, natures:['cp', 'deposit'], image_url:'https://lh3.googleusercontent.com/i904_m1OrYim9EZyNZiQ4PDGfeoV8NleF91S1_pBrvGLqI_pKfiNdinc-htQa15MAXxgG5NXIoB4h7jIzUfXvEVvD9X-lUZgM45paGcCPi83H81NhUvog3mOwzGVKDJeOXuG1hdiQw=w126-h107'},
{parent

In [44]:
import_family_group('Microstructure', filter_observation='Cross-section', fieldsets=[], filter_print=FAMILY|CHARACTERISTIC, shared_sub_characteristics=True) #FAMILY|CHARACTERISTI



// new family_name=Microstructure
{fg:'fgMicrostructure', order:1, observation:cross_section, uid:'microstructureMicrostructureCS', name:'Microstructure', visible: False, variable: False, description:'Assessed on the observed area. The blank option corresponds to a stratum which microstructure is not known., natures:['M']'},
{family_uid:'microstructureMicrostructureCS', uid:'microstructureBlankCS', name:'Blank', order:1, natures:['M']},
{family_uid:'microstructureMicrostructureCS', uid:'microstructureDeformedDendritesCS', name:'Deformed dendrites', order:2, natures:['M'], description:'Stratum containing crystals in the form of deformed trees with branches.'},
{family_uid:'microstructureMicrostructureCS', uid:'microstructureDendritesCS', name:'Dendrites', order:3, natures:['M'], description:'Stratum containing crystals in the form of trees with branches.'},
{family_uid:'microstructureMicrostructureCS', uid:'microstructureElongatedGrainsCS', name:'Elongated grains', order:4, natures:['M'

In [185]:
print('\n //FAMILY \n')
import_family_group('Microstructure', filter_observation='Cross-section', fieldsets=[], filter_print=FAMILY, shared_sub_characteristics=True) #FAMILY|CHARACTERISTI
print('\n //CHARACTERISTICS \n')
import_family_group('Microstructure', filter_observation='Cross-section', fieldsets=[], filter_print=CHARACTERISTIC, shared_sub_characteristics=True) #FAMILY|CHARACTERISTI
print('\n //SUBCHARACTERISTICS \n')
import_family_group('Microstructure', filter_observation='Cross-section', fieldsets=[], filter_print=SUBCHARACTERISTIC, shared_sub_characteristics=True) #FAMILY|CHARACTERISTI



 //FAMILY 



UnboundLocalError: local variable 'family_name' referenced before assignment

In [236]:
import csv
csv_file_path='./20200608 SEARCH-by strat. representation-Bin. obs_v1 - MiCorr.csv'
with open(csv_file_path,"r") as f:
    reader =csv.DictReader(f)
    print(reader.fieldnames)

csv_file_path='./Copy20210102 of 20200608 SEARCH-by strat. representation-Bin. obs_v1 - MiCorr.csv'
with open(csv_file_path,"r") as f:
    reader =csv.DictReader(f)
    print(reader.fieldnames)

def to_uid(name):
    camel_case_name=''.join(name.title().split(' '))

    return camel_case_name[0].lower() + camel_case_name[1:]


['Characteristic', 'Sub-characteristic', 'Observation mode', 'Values', 'Sub-values', 'Image', 'Variables', 'unit', 'uid', 'Definition from observation under binocular', 'Definition from observation on cross-section', 'Notice for modification of the current MiCorr interface', 'Comments', 'CP Binocular', 'CP CS', 'CM Binocular', 'CM CS', 'D Binocular', 'D CS', 'M Binocular', 'M CS', 'NMM Binocular', 'NMM CS', 'POM Binocular', 'POM CS', 'S Binocular', 'S CS', 'SV Binocular', 'SV CS']
['Characteristic', 'Sub-characteristic', 'Observation mode', 'Values', 'Sub-values', 'Image', 'Variables', 'unit', 'uid', 'Definition from observation under binocular', 'Definition from observation on cross-section', 'Notice for modification of the current MiCorr interface', 'Comments', 'CP Binocular', 'CP CS', 'CM Binocular', 'CM CS', 'D Binocular', 'D CS', 'M Binocular', 'M CS', 'NMM Binocular', 'NMM CS', 'POM Binocular', 'POM CS', 'S Binocular', 'S CS', 'SV Binocular', 'SV CS']


## Interface cross-section special case

In [59]:
# we treat Roughness as a fieldset (as it includes multiple families instead of characteristics)
# to do add group/fieldset column to spreadsheet

import_family_group('Interface', filter_observation='Cross-section', fieldsets=['Roughness'], filter_print=CHARACTERISTIC) #FAMILY|CHARACTERISTIC

{family_uid:'interfaceProfileCSFamily', uid:'interfaceBlankCSCharacteristic', name:'Blank', order:1, natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv']},
{family_uid:'interfaceProfileCSFamily', uid:'interfaceStraightCSCharacteristic', name:'Straight', order:2, natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv'], description:'No waves along the observed length.'},
{family_uid:'interfaceProfileCSFamily', uid:'interfaceWavyCSCharacteristic', name:'Wavy', order:3, natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv'], description:'Shows variations or undulations with a low height to length ratio. Angles are greater than 90°.'},
{family_uid:'interfaceProfileCSFamily', uid:'interfaceBumpyCSCharacteristic', name:'Bumpy', order:4, natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil', 'sv'], description:'Presents variations that are close in height and length. Angles can be equal to or less than 90°.'},
{family_uid:'interfaceProfileCSFamily', uid:'interfaceIrr

# we treat Roughness as a fieldset (as it includes multiple families instead of characteristics)
# to do add group/fieldset column to spreadsheet

import_family_group('Interface', filter_observation='Cross-section', fieldsets=['Roughness'], filter_print=CHARACTERISTIC) #FAMILY|CHARACTERISTIC

In [ ]:
# we treat Roughness as a fieldset (as it includes multiple families instead of characteristics)
# to do add group/fieldset column to spreadsheet

import_family_group('Interface', filter_observation='Cross-section', fieldsets=['Roughness'], filter_print=CHARACTERISTIC) #FAMILY|CHARACTERISTIC

In [ ]:
# we treat Roughness as a fieldset (as it includes multiple families instead of characteristics)
# to do add group/fieldset column to spreadsheet

import_family_group('Microstructure', filter_observation='Cross-section', fieldsets=[], filter_print=CHARACTERISTIC) #FAMILY|CHARACTERISTIC

# Composition


In [256]:
print('\n //FAMILY \n')
import_family_group('Composition',filter_observation='Binocular', fieldsets=['Second band / aggregate'], filter_print=FAMILY|DEBUG)
print('\n //CHARACTERISTIC \n')
import_family_group('Composition',filter_observation='Binocular', fieldsets=['Second band / aggregate'], filter_print=CHARACTERISTIC|DEBUG)
print('\n //SUBCHARACTERISTIC \n')
import_family_group('Composition',filter_observation='Binocular', fieldsets=['Second band / aggregate'], filter_print=SUBCHARACTERISTIC|DEBUG)




 //FAMILY 


OrderedDict([('Characteristic', 'Composition'), ('Sub-characteristic', 'Main element(s)'), ('Observation mode', 'Binocular'), ('Values', ''), ('Sub-values', ''), ('Image', ''), ('Variables', 'list'), ('unit', 'elementFamily'), ('uid', ''), ('Definition from observation under binocular', 'Determined in the observed area. Only the family element(s) should be given.'), ('Definition from observation on cross-section', ''), ('Notice for modification of the current MiCorr interface', ''), ('Comments', 'Only if elementary analysis has been carried out.'), ('CP Binocular', 'x'), ('CP CS', ''), ('CM Binocular', ''), ('CM CS', ''), ('D Binocular', 'x'), ('D CS', ''), ('M Binocular', 'x'), ('M CS', ''), ('NMM Binocular', 'x'), ('NMM CS', ''), ('POM Binocular', 'x'), ('POM CS', ''), ('S Binocular', 'x'), ('S CS', ''), ('SV Binocular', ''), ('SV CS', '')])

// new family_name=Main element(s) uid: compositionMainElementsFamily
{fg:'fgComposition', order:1, observation:binocular, uid:'c

In [251]:
print('\n //FAMILY \n')
import_family_group('Composition',filter_observation='Binocular', fieldsets=[ 'Compounds', 'Second band / aggregate'], filter_print=FAMILY)



 //FAMILY 

// new family_name=Main element(s) uid: compositionMainElementsFamily
{fg:'fgComposition', order:1, observation:binocular, uid:'compositionMainElementsFamily', name:'Main element(s)', visible: False, variable: False, description:'Determined in the observed area. Only the family element(s) should be given.', natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil'], list_elem_family:'elementFamily', upto:0},
// new family_name=Secondary element(s) uid: compositionSecondaryElementsFamily
{fg:'fgComposition', order:2, observation:binocular, uid:'compositionSecondaryElementsFamily', name:'Secondary element(s)', visible: False, variable: False, description:'Any other element(s) having a concentration greater than 1% (in weight) and / or affecting the properties of the CP.', natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil'], list_elem_family:'elementFamily', upto:0},
// new fieldset=Compounds
{fg:'fgComposition', order:4, observation:binocular, uid:'compositionCompoundsXrd

In [274]:

print('\n //CHARACTERISTIC \n')
import_family_group('Composition',filter_observation='Cross-section', fieldsets=[ 'Main element(s)', 'Secondary element(s)','Compounds', 'Additional element(s)', 'Second band / aggregate'], filter_print=CHARACTERISTIC)
# print('\n //SUBCHARACTERISTIC \n')
# import_family_group('Composition',filter_observation='Cross-section', fieldsets=[ 'Main element(s)', 'Secondary element(s)','Compounds', 'Additional element(s)', 'Second band / aggregate'], filter_print=SUBCHARACTERISTIC)



 //CHARACTERISTIC 

// new fieldset=Main element(s)
// new fieldset=Secondary element(s)
// new fieldset=Compounds
// new fieldset=Additional element(s)
{family_uid:'compositionSecondBandAggregateCompoundsCSFamily', uid:'compositionSecondBandAggregateCompoundsTechniquesCSCharacteristic', name:'Techniques', order:1, natures:['cp']},
{family_uid:'compositionCSFamily', uid:'compositionCompositionEnamelCSCharacteristic', name:'Enamel', order:2, natures:['nmm'], description:'Opaque or semi-transparent glossy substance that is a type of glass, applied by vitrification to metallic surfaces for ornament or as a protective coating.'},
{family_uid:'compositionCSFamily', uid:'compositionCompositionLeatherCSCharacteristic', name:'Leather', order:3, natures:['nmm', 'pom'], description:'Organic material created by tanning of animals skin.'},
{family_uid:'compositionCSFamily', uid:'compositionCompositionNielloCSCharacteristic', name:'Niello', order:4, natures:['nmm'], description:'Black compound of 

In [272]:
print('\n //FAMILY \n')
import_family_group('Composition',filter_observation='Cross-section', fieldsets=[ 'Main element(s)', 'Secondary element(s)','Compounds', 'Additional element(s)', 'Second band / aggregate','Second band / aggregate_Compounds','Second band / aggregate_main element(s)',
                                                                                 'Second band / aggregate_secondary element(s)','Second band / aggregate_Additional element(s)'], filter_print=FAMILY)



 //FAMILY 

// new fieldset=Main element(s)
{fg:'fgComposition', order:2, observation:cross_section, uid:'compositionMainElementsListCSFamily', name:'List', visible: False, variable: False, description:'Determined in the observed area. Only the family element(s) should be given.', natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil'], fieldset:'Main element(s)', list_elem_family:'elementFamily', upto:0},
{fg:'fgComposition', order:3, observation:cross_section, uid:'compositionMainElementsAmountCSVarFamily', name:'Amount', visible: False, variable: True, description:'In % (mass).', natures:['cp', 'deposit', 'metal', 'nmm', 'pom', 'soil'], unit:'%', fieldset:'Main element(s)'},
// new fieldset=Secondary element(s)
{fg:'fgComposition', order:5, observation:cross_section, uid:'compositionSecondaryElementsListCSFamily', name:'List', visible: False, variable: False, description:'Any other element(s) having a concentration greater than 1% (in weight) and / or affecting the properties of t